# Lightweight Fine-Tuning Project

In order to fine tune a larger foundational model (such as BERT, or GPT-2) to a more specific task, you can take the pretrained version of the model and update its parameters using more specific data. This notebook compared the performance of a foundatitional model with its fine tuned version.


## Loading and Evaluating a Foundation Model

This step includes loading an appropriate tokenizer and dataset. Is this notebook we examine the GPT-2 model as it is fairly small, but powerful and compatible with PEFT. 

In [1]:
# TRAIN_TEST_SIZE = 25
TRAIN_TEST_SIZE = 250

First things first we load a toy dataset. We can use the imdb test set.

In [2]:
from datasets import load_dataset

# Load the train and test splits of the imdb dataset
splits = ["train", "test"]
ds = load_dataset("imdb")

# Thin out the dataset to make it run faster for this example
for split in splits:
    ds[split] = ds[split].shuffle(seed=42).select(range(TRAIN_TEST_SIZE))

ds.pop('unsupervised')
# Show the dataset
ds

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 250
    })
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 250
    })
})

Next we load the tokenizer that is used for gpt2. It needs to have the pad_token explicitly set.

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")
# GPT-2 Tokenizer doesn't have a padding token.
tokenizer.pad_token = tokenizer.eos_token

def preprocess_function(examples):
    """Preprocess the imdb dataset by returning tokenized examples."""
    tokens = tokenizer(examples['text'], padding='max_length', truncation=True)
    return tokens

# Let's use a lambda function to tokenize all the examples
tokenized_ds = {}
for split in splits:
    tokenized_ds[split] = ds[split].map(preprocess_function, batched=True)

# Inspect the available columns in the dataset
tokenized_ds

Map: 100%|██████████| 250/250 [00:00<00:00, 728.02 examples/s]


{'train': Dataset({
     features: ['text', 'label', 'input_ids', 'attention_mask'],
     num_rows: 250
 }),
 'test': Dataset({
     features: ['text', 'label', 'input_ids', 'attention_mask'],
     num_rows: 250
 })}

Load GPT-2 Model, also setting the pad token. The foundation model just creates text embeddings, so it needs a classification head added to it in order to make predictions for this particular dataset. In order to only train that classication head, we freeze the other layers.

In [4]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "gpt2",
    num_labels=2,
    id2label={0: "NEGATIVE", 1: "POSITIVE"},  # For converting predictions to strings
    label2id={"NEGATIVE": 0, "POSITIVE":1},
)
model.config.pad_token_id = model.config.eos_token_id

# Freeze all the layers so only the classification head is trained.
for param in model.base_model.parameters():
    param.requires_grad = False

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

training_args = TrainingArguments(
        output_dir="./data/sentiment_analysis",
        learning_rate=2e-3,
        # Reduce the batch size if you don't have enough memory
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=5,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    )

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.661136,0.652000
2,No log,0.517181,0.748000
3,No log,0.747950,0.596000
4,No log,0.606842,0.688000
5,No log,0.534791,0.716000


TrainOutput(global_step=315, training_loss=0.5692306760757688, metrics={'train_runtime': 235.4579, 'train_samples_per_second': 5.309, 'train_steps_per_second': 1.338, 'total_flos': 653241876480000.0, 'train_loss': 0.5692306760757688, 'epoch': 5.0})

In [6]:
trainer.evaluate()

{'eval_loss': 0.5171810984611511,
 'eval_accuracy': 0.748,
 'eval_runtime': 21.9418,
 'eval_samples_per_second': 11.394,
 'eval_steps_per_second': 2.871,
 'epoch': 5.0}

We can also see what the model predicts for a fake review.

In [7]:
def get_sentiment(sentence, model):
    inputs = tokenizer(sentence, return_tensors="pt")
    outputs = model(**inputs.to(device='cuda'))
    prediction = outputs.logits.argmax(-1).item()
    return "positive" if prediction == 1 else "negative"


sentence = "I did not like this movie!"
print(get_sentiment(sentence, model))

negative


In [8]:
model.save_pretrained("gpt")

## Performing Parameter-Efficient Fine-Tuning

### PEFT technique

Parameter-efficient fine-tuning (PEFT) is a method of updating a predefined subset of a model's parameters to tailor it to specific tasks, without the need to modify the entire model, thus saving computational resources.

A specific PEFT technique is Low Rank Adaption (LoRA). This involves approximating a large matrices using two smaller thin matricies, greatly reducing the amount of parameters that have to be updated.

In [9]:
from peft import LoraConfig, TaskType
from peft import get_peft_model
from peft import AutoPeftModelForSequenceClassification

model_loaded = AutoModelForSequenceClassification.from_pretrained("gpt")

lora_config = LoraConfig(
fan_in_fan_out=True,
task_type=TaskType.SEQ_CLS
)

lora_model = get_peft_model(model_loaded, lora_config)
lora_model.config.pad_token_id = lora_model.config.eos_token_id

Lets look at which layers of the model that we will be retraining.

In [10]:
lora_model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): GPT2ForSequenceClassification(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): Linear(
                in_features=768, out_features=2304, bias=True
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2304, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()

In [11]:
# LoRA takes in "labels", not "label" so we need to rename the 
# training and testing sets
train_lora = tokenized_ds['train'].rename_column('label', 'labels')
test_lora = tokenized_ds['test'].rename_column('label', 'labels')

In [12]:
trainer_lora = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=train_lora,
    eval_dataset=test_lora,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)
trainer_lora.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.463424,0.820000
2,No log,0.967935,0.664000
3,No log,0.854051,0.816000
4,No log,0.761409,0.868000
5,No log,0.868605,0.868000


TrainOutput(global_step=315, training_loss=0.32488262842571924, metrics={'train_runtime': 379.2005, 'train_samples_per_second': 3.296, 'train_steps_per_second': 0.831, 'total_flos': 655518597120000.0, 'train_loss': 0.32488262842571924, 'epoch': 5.0})

In [13]:
trainer_lora.evaluate()

{'eval_loss': 0.4634243845939636,
 'eval_accuracy': 0.82,
 'eval_runtime': 22.6826,
 'eval_samples_per_second': 11.022,
 'eval_steps_per_second': 2.777,
 'epoch': 5.0}

In [14]:
lora_model.save_pretrained("gpt-lora")

## Performing Inference with a PEFT Model

Evaluate the performance of the trained PEFT model. The results can then be compared to foundational model.

In [15]:
from peft import AutoPeftModelForSequenceClassification
lora_model_loaded = AutoPeftModelForSequenceClassification.from_pretrained("gpt-lora")

In [16]:
# now we put the loaded model into a trainer so we can use the evaluate method and 
# compare it to the foundational model
trainer_lora_loaded = Trainer(
    model=lora_model_loaded,
    args=training_args,
    train_dataset=train_lora,
    eval_dataset=test_lora,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

In [17]:
trainer_lora_loaded.evaluate()

{'eval_loss': 0.4634243845939636,
 'eval_accuracy': 0.82,
 'eval_runtime': 22.6565,
 'eval_samples_per_second': 11.034,
 'eval_steps_per_second': 2.781}

Awesome, this is the same result that we had before saving and loading the model. Comparing this to the foundational model without finetuning we have: 0.748 (foundational) compared to 0.82. This is a significant improvement especially considering the small size of the train set.

In [19]:
sentence = "I did not like this movie!"
print(get_sentiment(sentence, lora_model_loaded.to(device='cuda')))

negative
